In [1]:
%reload_ext autoreload

In [2]:
%autoreload 2

In [16]:
def train_ncRNA_model(fname=None, model_fname=None, n_iter=40, verbose=False):
    #parameters
    times=2
    size=200
    active_set_size=400
    threshold=1
    n_active_learning_iterations=3
    train_test_split=0.7
    
    
    def rfam_uri(family_id):
        return 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id)
    
    def pre_processor( data, **args):
        #from eden.converter.rna.rnashapes import rnashapes_to_eden
        #graphs = rnashapes_to_eden( data, **args )
        from eden.converter.rna.rnafold import rnafold_to_eden
        graphs = rnafold_to_eden( data, **args )
        return graphs

    
    from eden.graph import Vectorizer
    vectorizer = Vectorizer()

    from sklearn.linear_model import SGDClassifier
    estimator = SGDClassifier(class_weight='auto', shuffle=True)
    

    #create iterable from files
    from eden.converter.fasta import fasta_to_sequence
    seqs = fasta_to_sequence( rfam_uri( rfam_id ) )
    from itertools import tee
    seqs,seqs_=tee(seqs)
    iterable_pos = seqs
    from eden.modifier.seq import seq_to_seq, shuffle_modifier
    iterable_neg = seq_to_seq( seqs_, modifier=shuffle_modifier, times=times, order=2 )

    #consier only first 'size' elements
    from itertools import islice
    iterable_pos = islice(iterable_pos,size)
    iterable_neg = islice(iterable_neg,size*times)

    #split train/test
    from eden.util import random_bipartition_iter
    iterable_pos_train, iterable_pos_test = random_bipartition_iter(iterable_pos, relative_size=train_test_split)
    iterable_neg_train, iterable_neg_test = random_bipartition_iter(iterable_neg, relative_size=train_test_split)
    
    #make predictive model
    from eden.model import ActiveLearningBinaryClassificationModel
    model = ActiveLearningBinaryClassificationModel( pre_processor, estimator=estimator, vectorizer=vectorizer )

    #optimize hyperparameters and fit model
    from numpy.random import randint
    from numpy.random import uniform
#    pre_processor_parameters={'max_num':[1,2,3], 
#                              'shape_type':[3,4,5], 
#                              'energy_range':randint(10, 40, size=n_iter)}
    pre_processor_parameters={} 

    vectorizer_parameters={'complexity':[1,2]}

    estimator_parameters={'n_iter':randint(5, 100, size=n_iter),
                          'penalty':['l1','l2','elasticnet'],
                          'l1_ratio':uniform(0.1,0.9, size=n_iter), 
                          'loss':['hinge', 'log', 'modified_huber', 'squared_hinge', 'perceptron'],
                          'power_t':uniform(0.1, size=n_iter),
                          'alpha': [10**x for x in range(-8,0)],
                          'eta0': [10**x for x in range(-4,-1)],
                          'learning_rate': ["invscaling", "constant", "optimal"]}

    model.optimize(iterable_pos_train, iterable_neg_train, 
                   n_active_learning_iterations=n_active_learning_iterations,
                   size_positive=-1,
                   size_negative=active_set_size,
                   n_iter=n_iter, cv=3, n_jobs=1, verbose=verbose,
                   pre_processor_parameters=pre_processor_parameters, 
                   vectorizer_parameters=vectorizer_parameters, 
                   estimator_parameters=estimator_parameters)

    #save model
    model.save(model_fname)
    print 'Saved model in %s'%model_fname
    
    #estimate predictive performance
    model.estimate( iterable_pos_test, iterable_neg_test, cv=5 )
    
    
    
    
def test_ncRNA_model(fname=None, model_fname=None):
    from eden.model import ActiveLearningBinaryClassificationModel

    model = ActiveLearningBinaryClassificationModel()
    model.load(model_fname)

    def rfam_uri(family_id):
        return 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id)

    from eden.converter.fasta import fasta_to_sequence
    seqs = fasta_to_sequence( rfam_uri( rfam_id ) )
    from itertools import tee
    seqs,seqs_=tee(seqs)
    
    predictions= model.decision_function( seqs_ )
    
    from itertools import izip
    seqs,seqs_=tee(seqs)
    results = [(p,s) for s,p in izip(seqs_,predictions)]
    
    return results

In [4]:
def rfam_url(family_id):
    return 'http://rfam.xfam.org/family/%s/alignment?acc=%s&format=fastau&download=0'%(family_id,family_id)

In [5]:
import requests
import numpy

def rfam_size(rfam_id):
    uri = rfam_url( rfam_id )
    from eden.modifier.fasta import fasta_to_fasta, one_line_modifier
    iterable = fasta_to_fasta( uri , modifier = one_line_modifier, header_only = True)
    size = sum(1 for x in iterable)
    return size

def rfam_stats(rfam_id):
    uri = rfam_url( rfam_id )
    from eden.modifier.fasta import fasta_to_fasta, one_line_modifier
    iterable = fasta_to_fasta( uri , modifier = one_line_modifier, sequence_only = True)
    len_seqs = [len(seq) for seq in iterable]
    median = numpy.percentile(len_seqs, 50)
    lower_quartile = numpy.percentile(len_seqs, 25)
    upper_quartile = numpy.percentile(len_seqs, 75)
    return lower_quartile, median, upper_quartile 

In [6]:
%%time
#identify non available rfam families

blacklist = []

size_limit = 1000
num = 2600
rfam_ids = ['RF%0.5d'%i for i in range(1,num)]
print 'Determining non valid Rfam family accession or ID...'
counter = 0
for rfam_id in rfam_ids:
    try:
        size = rfam_size( rfam_id )
    except:
        counter += 1
        blacklist.append(rfam_id)
        print counter, rfam_id
    else:
        if size > size_limit:
            counter += 1
            blacklist.append(rfam_id)
            print counter, rfam_id, '>>'
    finally:
        pass
    
arfam_ids = [rfam_id for rfam_id in rfam_ids if rfam_id not in blacklist]
print 'Tried %d families, found %d non valid Rfam family accession or ID' % (num, len(blacklist))

Determining non valid Rfam family accession or ID...
Tried 2600 families, found 0 non valid Rfam family accession or ID
CPU times: user 8.27 s, sys: 2.39 s, total: 10.7 s
Wall time: 6min 30s


In [7]:
%%time
#select families with num instances within specified limits
size_upper_limit = 300
size_lower_limit = 150
selected_rfam_ids = [rfam_id for rfam_id in arfam_ids if rfam_size( rfam_id ) >= size_lower_limit and rfam_size( rfam_id ) <= size_upper_limit]
print 'Rfam families stisfying the num seqs limits > %d and < %d are: %d' % (size_lower_limit, size_upper_limit, len(selected_rfam_ids) )

Rfam families stisfying the num seqs limits > 150 and < 300 are: 22
CPU times: user 7.7 s, sys: 2.2 s, total: 9.9 s
Wall time: 7min 39s


In [8]:
%%time
#print stats
rfam_legths = {}
print 'Rfam statistics:'
for i,rfam_id in enumerate(selected_rfam_ids):
    size = rfam_size( rfam_id )
    lower_quartile, median, upper_quartile = rfam_stats( rfam_id )
    print "%0.5d rfam: %s num seqs: %0.4d length stats: Q1: %0.4d Q2: %0.4d Q3: %0.4d" % ( i+1, rfam_id, size, lower_quartile, median, upper_quartile )
    rfam_legths[rfam_id] = median

Rfam statistics:
00001 rfam: RF00004 num seqs: 0208 length stats: Q1: 0191 Q2: 0193 Q3: 0195
00002 rfam: RF00015 num seqs: 0170 length stats: Q1: 0135 Q2: 0141 Q3: 0146
00003 rfam: RF00020 num seqs: 0180 length stats: Q1: 0115 Q2: 0116 Q3: 0119
00004 rfam: RF00026 num seqs: 0188 length stats: Q1: 0103 Q2: 0107 Q3: 0108
00005 rfam: RF00169 num seqs: 0261 length stats: Q1: 0081 Q2: 0094 Q3: 0098
00006 rfam: RF00380 num seqs: 0157 length stats: Q1: 0165 Q2: 0168 Q3: 0172
00007 rfam: RF00386 num seqs: 0160 length stats: Q1: 0090 Q2: 0090 Q3: 0091
00008 rfam: RF01051 num seqs: 0155 length stats: Q1: 0085 Q2: 0087 Q3: 0089
00009 rfam: RF01055 num seqs: 0160 length stats: Q1: 0135 Q2: 0142 Q3: 0147
00010 rfam: RF01234 num seqs: 0160 length stats: Q1: 0136 Q2: 0138 Q3: 0138
00011 rfam: RF01699 num seqs: 0194 length stats: Q1: 0167 Q2: 0168 Q3: 0172
00012 rfam: RF01701 num seqs: 0265 length stats: Q1: 0066 Q2: 0068 Q3: 0069
00013 rfam: RF01705 num seqs: 0201 length stats: Q1: 0076 Q2: 0080 Q3: 

In [18]:
%%time
for rfam_id in selected_rfam_ids:
    print rfam_id
    model_fname=rfam_id+'.model'
    train_ncRNA_model(fname=rfam_id, model_fname=model_fname, n_iter=10, verbose=False)
    results = test_ncRNA_model(fname=rfam_id, model_fname=model_fname)

RF00004
Saved model in RF00004.model
Classifier:
SGDClassifier(alpha=0.0001, class_weight='auto', epsilon=0.1, eta0=0.0001,
       fit_intercept=True, l1_ratio=0.10158070470654268,
       learning_rate='constant', loss='squared_hinge', n_iter=87, n_jobs=1,
       penalty='l2', power_t=0.48091437952204807, random_state=None,
       shuffle=True, verbose=0, warm_start=False)
--------------------------------------------------------------------------------
Predictive performance:
            accuracy: 0.856 +- 0.087
           precision: 0.738 +- 0.120
              recall: 0.917 +- 0.129
                  f1: 0.812 +- 0.109
   average_precision: 0.967 +- 0.046
             roc_auc: 0.978 +- 0.031
--------------------------------------------------------------------------------
RF00015
Saved model in RF00015.model
Classifier:
SGDClassifier(alpha=0.01, class_weight='auto', epsilon=0.1, eta0=0.001,
       fit_intercept=True, l1_ratio=0.64640302916124981,
       learning_rate='invscaling', los